In [1]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from utils import *
from vehicle import Vehicle

import networkx as nx
from bokeh.io import show, output_file, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx


## Estrazione dati con un piccolo parser dei dati

In [2]:
path='Instances/A1.txt'

In [3]:
#support
count = 0
#first 3 lines of the txt file
number_of_customers=0
one=0
number_of_vehicles=0
#vettori di informazione
vect_cord=[] #lista/vettore di coppiadi coordinate
vect_domanda=[] #linehaul
vect_carico=[] #backhaul
vec_b=[] #vettore delle istanze backhaul
vec_l=[] #vettore delle istanze linehaul
with open(path) as file:
    for line in file:
        #print(line)
        count+=1
        if count  == 1:
            number_of_customers=int((line.split())[0])
        if count  == 2:
            one=int((line.split())[0])
        if count == 3:
            number_of_vehicles=int((line.split())[0])
        if count > 3:
            vect_cord.append((int((line.split())[0]),int((line.split())[1])))
            vect_domanda.append((int((line.split())[2])))
            vect_carico.append((int((line.split())[3]))) 
            if int((line.split())[3]) == 0:
                vec_l.append(count-4) #il numero delle istanze linehaul
            if int((line.split())[3]) > 0:
                vec_b.append(count-4) # il numero delle istanze backhaul

#capacità massima di ogni mezzo
capacity=vect_carico[0]                     

In [4]:
df_data = pd.DataFrame({'': [number_of_customers, number_of_vehicles, capacity]},
                       index=['Number of customers', 'Number of Vehicles', 'Capacity'])

In [5]:
df_instances = pd.DataFrame({'Cordinate X': [ x[0] for x in vect_cord],
                             'Cordinate Y': [ x[1] for x in vect_cord], 
                             'Linehaul': vect_domanda, 
                             'Backhaul' : vect_carico,
                            })

In [6]:
df_data

,
Number of customers,25
Number of Vehicles,8
Capacity,1550


all'indice 0 è riportato il magazzino. dove il campo Backhaul rappresenta la capacità massima

In [7]:
df_instances

,Cordinate X,Cordinate Y,Linehaul,Backhaul
0,12000,16000,0,1550
1,9795,21510,0,549
2,7758,10895,0,558
3,12927,2464,0,851
4,23286,5538,0,388
5,7416,15826,0,194
6,21524,24879,483,0
7,22337,17915,389,0
8,7642,9722,435,0
9,3707,20036,452,0


In [8]:
dict_coord = {}
for index, value in enumerate(vect_cord):
    dict_coord[index]=(value[0]/10000, value[1]/10000)

In [9]:
dict_coord

{0: (1.2, 1.6),
 1: (0.9795, 2.151),
 2: (0.7758, 1.0895),
 3: (1.2927, 0.2464),
 4: (2.3286, 0.5538),
 5: (0.7416, 1.5826),
 6: (2.1524, 2.4879),
 7: (2.2337, 1.7915),
 8: (0.7642, 0.9722),
 9: (0.3707, 2.0036),
 10: (1.5492, 0.2721),
 11: (1.243, 1.0638),
 12: (1.3595, 0.1918),
 13: (0.429, 0.2721),
 14: (1.8684, 0.5268),
 15: (0.8686, 0.9546),
 16: (2.314, 1.0989),
 17: (0.5263, 1.4289),
 18: (1.9557, 1.7564),
 19: (2.3281, 2.9684),
 20: (0.0374, 2.2683),
 21: (1.8804, 0.9082),
 22: (0.5444, 2.5004),
 23: (2.039, 2.5745),
 24: (0.431, 2.1692),
 25: (0.8526, 0.2495)}

In [10]:
import networkx as nx
from bokeh.io import show, output_file
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4
from bokeh.io import output_notebook
G=nx.Graph()
for e in list(dict_coord.keys()):
    G.add_node(e,pos=dict_coord[e])

In [11]:
pos=nx.get_node_attributes(G,'pos')

In [12]:
plot = Plot(plot_width=500, plot_height=500,
            x_range=Range1d(-1.3, 3.3), y_range=Range1d(-1.3, 3.3))
plot.title.text = "Graph Interaction"

node_hover_tool = HoverTool(tooltips=[("Magazzino", "@index"),("Coordinate", '@pos')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

Loading BokehJS ...

## Calcolo dei Dati raccolti

In [13]:
distance = distance_matrix(vect_cord)

In [14]:
dict_distance={}
for c, i in enumerate(distance):
    for k,j in enumerate(i):
        dict_distance[(c,k)]=j

In [15]:
dict_distance

{(0, 0): 0.0,
 (0, 1): 5934.823080766603,
 (0, 2): 6637.438436625985,
 (0, 3): 13567.705222328497,
 (0, 4): 15389.192311489254,
 (0, 5): 4587.301167353196,
 (0, 6): 13020.876199396107,
 (0, 7): 10512.88704400461,
 (0, 8): 7642.345713195655,
 (0, 9): 9222.968339965177,
 (0, 10): 13730.473589792888,
 (0, 11): 5379.214068988145,
 (0, 12): 14172.041102113697,
 (0, 13): 15354.997264734371,
 (0, 14): 12643.246418542984,
 (0, 15): 7255.11626371349,
 (0, 16): 12215.143101904292,
 (0, 17): 6950.876923093949,
 (0, 18): 7717.146169407445,
 (0, 19): 17734.509212267476,
 (0, 20): 13409.93530931451,
 (0, 21): 9703.25409334415,
 (0, 22): 11137.915065217547,
 (0, 23): 12859.126136717066,
 (0, 24): 9567.39065785442,
 (0, 25): 13944.665682618568,
 (1, 0): 5934.823080766603,
 (1, 1): 0.0,
 (1, 2): 10808.681418193433,
 (1, 3): 19301.801470329137,
 (1, 4): 20907.220403487405,
 (1, 5): 6161.777097558788,
 (1, 6): 12203.261940972996,
 (1, 7): 13047.060550177577,
 (1, 8): 11983.002670449507,
 (1, 9): 6263.898

In [16]:
from collections import Counter
c = Counter(dict_distance) #inizializzo una variabile della classe Counter 
dict_distance_Order=c.most_common() #ordino per i valori dei saving in maniera decrescente
dict_distance = {} # svuoto il vecchio dizionario per questioni di memoria
for e in dict_distance_Order: # ciclo su ogni valore del vettore della classe counter che è composto da copie 
    dict_distance[e[0]]=e[1] # reinserisco su K il nome delle istanze e su V il valore a loro associato
   

In [17]:
for e in dict_distance:
    if 0 in e:
        print(e)

(0, 19)
(19, 0)
(0, 4)
(4, 0)
(0, 13)
(13, 0)
(0, 12)
(12, 0)
(0, 25)
(25, 0)
(0, 10)
(10, 0)
(0, 3)
(3, 0)
(0, 20)
(20, 0)
(0, 6)
(6, 0)
(0, 23)
(23, 0)
(0, 14)
(14, 0)
(0, 16)
(16, 0)
(0, 22)
(22, 0)
(0, 7)
(7, 0)
(0, 21)
(21, 0)
(0, 24)
(24, 0)
(0, 9)
(9, 0)
(0, 18)
(18, 0)
(0, 8)
(8, 0)
(0, 15)
(15, 0)
(0, 17)
(17, 0)
(0, 2)
(2, 0)
(0, 1)
(1, 0)
(0, 11)
(11, 0)
(0, 5)
(5, 0)
(0, 0)


In [18]:
saving=saving_matrix(distance)

In [19]:
dict_Saving_Order=savingOrder(saving)

## Mappa dei saving ordinati

In [20]:
dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)

In [21]:
dict_l

{(10, 12): 25842.55886713752,
 (19, 23): 25707.57333546077,
 (6, 19): 25639.227282754746,
 (13, 25): 25057.6384306864,
 (6, 23): 24453.148454315884,
 (12, 25): 23014.972765793216,
 (10, 14): 22290.085230347024,
 (12, 14): 20722.63448605837,
 (10, 25): 20705.474144157444,
 (12, 13): 20187.454114859116,
 (20, 22): 18971.83557899932,
 (20, 24): 18918.486601044668,
 (14, 21): 18530.613196989063,
 (9, 20): 18376.673157400754,
 (10, 13): 17883.47085452726,
 (14, 16): 17606.784387788714,
 (22, 24): 17204.548662092155,
 (16, 21): 17181.568614858395,
 (9, 24): 17027.98985374109,
 (6, 7): 16522.467714042396,
 (7, 19): 16440.597494541624,
 (10, 21): 16262.14034141136,
 (14, 25): 16058.216331577321,
 (7, 16): 15755.635650471915,
 (7, 18): 15427.962372052043,
 (7, 23): 15303.574112351635,
 (8, 13): 15235.26174929038,
 (9, 22): 15097.976128781207,
 (12, 21): 15017.723462753673,
 (10, 16): 14682.769449547439,
 (13, 15): 14491.896913944403,
 (8, 25): 14306.147092724928,
 (15, 25): 14146.966834619574,


## Inizializzo la lista delle rotte

In [22]:
list_route = []
set_iniziale = {0}
set_first = {0}
set_visited = {0}
lista_keys= list(dict_l.keys())
for k in lista_keys:
    if len(list_route) < number_of_vehicles and k[0] not in set_iniziale and k[1] not in set_iniziale:
        list_route.append(Vehicle([0, k[0], k[1]], vect_domanda[k[0]]+vect_domanda[k[1]],
                         distance[0][k[0]]+distance[k[0]][k[1]]))
        set_iniziale.add(k[0])
        set_iniziale.add(k[1])
        set_first.add(k[0])
        set_visited.add(k[0])
        set_visited.add(k[1])

In [23]:
remove_first_instance(dict_l, set_first)

In [24]:
for e in list_route:
    print(e.route)

[0, 10, 12]
[0, 19, 23]
[0, 13, 25]
[0, 20, 22]
[0, 14, 21]
[0, 9, 24]
[0, 6, 7]
[0, 8, 15]


In [25]:
dict_l

{(12, 25): 23014.972765793216,
 (22, 24): 17204.548662092155,
 (16, 21): 17181.568614858395,
 (7, 16): 15755.635650471915,
 (7, 18): 15427.962372052043,
 (7, 23): 15303.574112351635,
 (12, 21): 15017.723462753673,
 (15, 25): 14146.966834619574,
 (12, 16): 13219.418030516883,
 (16, 18): 12444.398112914734,
 (12, 15): 12356.067850228745,
 (18, 23): 12352.973089873167,
 (21, 25): 11440.305226018903,
 (11, 12): 10753.77683921605,
 (7, 21): 10702.783265387883,
 (11, 25): 10293.39443795939,
 (16, 23): 10064.204748420127,
 (16, 25): 9256.642453920773,
 (17, 24): 9054.179023060005,
 (22, 23): 9032.68364806682,
 (18, 21): 8905.041609118736,
 (11, 15): 8734.330332701637,
 (17, 25): 8658.483931341456,
 (11, 21): 8521.293572367478,
 (15, 17): 8356.807089579892,
 (17, 22): 7372.263352601143,
 (11, 16): 6878.607033827835,
 (15, 21): 6829.736688097151,
 (7, 12): 6455.101679582756,
 (12, 17): 6207.69543216533,
 (23, 24): 5843.597600412697,
 (21, 23): 5824.071922469317,
 (12, 18): 5145.747959360691,
 (

In [26]:
for e in dict_l:
    if 9 in e:
        print(e)

In [27]:
a=vect_domanda[10]+vect_domanda[12]+vect_domanda[14]+vect_domanda[21]+vect_domanda[16]
b=vect_domanda[19]+vect_domanda[23]+vect_domanda[6]+vect_domanda[7]+vect_domanda[18]
c=vect_domanda[13]+vect_domanda[25]+vect_domanda[8]+vect_domanda[15]+vect_domanda[11]
d=vect_domanda[20]+vect_domanda[22]+vect_domanda[24]+vect_domanda[9]+vect_domanda[17]

In [28]:
print(a)
print(b)
print(c)
print(d)

2330
2297
2738
2684


In [29]:
c = 0
temp = dict_l.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_l.keys())
    flag = True
    for k in lista_keys:
        #if 17 in k and list_route[c].route[-1] in k: #controllo cosa c'è quando arriva a 17
            #print(f'La route {c}: \t\t {list_route[c].route} \t\t capacità raggiunta: {list_route[c].weight} \t\t pair {k}')
            #print('----')
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_domanda[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_domanda[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])

                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [30]:
len(dict_l)

66

In [31]:
for e in list_route:
    e.delivery = e.weight
    e.weight = 0
    print(e.route)

[0, 10, 12, 16]
[0, 19, 23, 18]
[0, 13, 25, 11]
[0, 20, 22]
[0, 14, 21]
[0, 9, 24, 17]
[0, 6, 7]
[0, 8, 15]


In [32]:
for e in dict_merge:
    if 17 in e:
        print(e)

(2, 17)
(5, 17)
(3, 17)
(1, 17)
(4, 17)


In [33]:
a=vect_carico[4]+vect_carico[5]
b=vect_carico[3]
c=vect_carico[2]
d=vect_carico[1]

In [34]:
print(a)
print(b)
print(c)
print(d)

582
851
558
549


In [35]:
c = 0

while(c < len(list_route)):
    if len(set_visited) <= number_of_customers:
        merge = list(dict_merge.keys())
        back = list(dict_b.keys())
        for k in merge:
            if list_route[c].route[-1] in k and different(list_route[c].route[-1], k) not in set_visited:
                    list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                    list_route[c].add_route(k)
                    set_visited.add(list_route[c].route[-1])
                    list_route[c].add_cost(distance[k[0]][k[1]]) 
                    break
    c += 1

In [36]:

c = 0
temp = dict_b.copy()
set_first = {0}
i = 0 
while(i < np.math.factorial(number_of_customers)/(np.math.factorial(number_of_customers-2)*2)):
    lista_keys = list(dict_b.keys())
    flag = True
    for k in lista_keys:
        if list_route[c].route[-1] in k \
        and different(list_route[c].route[-1], k) not in set_visited \
        and list_route[c].weight + vect_carico[different(list_route[c].route[-1],k)] <= capacity:
                
                list_route[c].add_weight(vect_carico[different(list_route[c].route[-1],k)])
                list_route[c].add_route(k)           
                set_visited.add(list_route[c].route[-1])
                list_route[c].add_cost(distance[k[0]][k[1]])
                set_first.add(k[0])
                break

    #remove_first_instance(dict_l, set_first)
    c = (c + 1) % number_of_vehicles
    
    i += 1

In [37]:
for e in list_route:
    e.route.append(0)
    e.cost += distance[0][e.route[-1]]
    e.pickUp= e.weight

In [38]:
for e in list_route:
    print(e.route)

[0, 10, 12, 16, 4, 0]
[0, 19, 23, 18, 3, 0]
[0, 13, 25, 11, 2, 0]
[0, 20, 22, 1, 0]
[0, 14, 21, 5, 0]
[0, 9, 24, 17, 0]
[0, 6, 7, 0]
[0, 8, 15, 0]


In [39]:
for e in list_route:
    print(f'Mezzo {list_route.index(e)+1}:\t Rotta: {e.route}\t\t Costo: {e.cost}\t Delivery: {e.delivery}\t Pickup: {e.pickUp}')
    print('----------------------------------------------------------------------')

Mezzo 1:	 Rotta: [0, 10, 12, 16, 4, 0]		 Costo: 34411.150475094495	 Delivery: 1169	 Pickup: 388
----------------------------------------------------------------------
Mezzo 2:	 Rotta: [0, 19, 23, 18, 3, 0]		 Costo: 47335.28942283378	 Delivery: 1425	 Pickup: 851
----------------------------------------------------------------------
Mezzo 3:	 Rotta: [0, 13, 25, 11, 2, 0]		 Costo: 33306.57035600122	 Delivery: 1301	 Pickup: 558
----------------------------------------------------------------------
Mezzo 4:	 Rotta: [0, 20, 22, 1, 0]		 Costo: 24566.20431052641	 Delivery: 1196	 Pickup: 549
----------------------------------------------------------------------
Mezzo 5:	 Rotta: [0, 14, 21, 5, 0]		 Costo: 29694.241590953963	 Delivery: 1161	 Pickup: 194
----------------------------------------------------------------------
Mezzo 6:	 Rotta: [0, 9, 24, 17, 0]		 Costo: 18449.426041932045	 Delivery: 1488	 Pickup: 0
----------------------------------------------------------------------
Mezzo 7:	 Rotta

In [40]:
print(f'numero nodi visitati, compreso il magazzino: {len(set_visited)}\nlista dei nodi: {set_visited}')

numero nodi visitati, compreso il magazzino: 26
lista dei nodi: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}


In [41]:
tot_cost=sum([x.cost for x in list_route])

In [42]:
path_solutions ='RPA_Solutions/Detailed_Solution_A1.txt'

round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2)


-5.82

In [43]:
list_route[3].route

[0, 20, 22, 1, 0]

In [44]:
for i in range(len(list_route[3].route)-1):
    G.add_edge(list_route[3].route[i],list_route[3].route[i+1])

In [45]:
plot = Plot(plot_width=500, plot_height=500,
            x_range=Range1d(-1.3, 3.3), y_range=Range1d(-1.3, 3.3))
plot.title.text = "Graph Interaction"

node_hover_tool = HoverTool(tooltips=[("Magazzino", "@index"),("Coordinate", '@pos')])
plot.add_tools(node_hover_tool, BoxZoomTool(), ResetTool())

graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.edge_renderer.glyph = MultiLine( line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

output_notebook()
show(plot)

Loading BokehJS ...